
---

# 🧩 Pydantic Types

### 🎯 Intent

Use Pydantic’s **built-in types** for common real-world data like emails, URLs, UUIDs, and payment cards — so you don’t reinvent validation.

---

### 🧩 Core Components

1. **📧 EmailStr**

   * Ensures valid email format.
   * Rejects invalid addresses automatically.

2. **🔗 AnyUrl & HttpUrl**

   * `AnyUrl` → accepts any valid URL.
   * `HttpUrl` → restricted to HTTP/HTTPS only.

3. **🆔 UUID**

   * Parses and validates UUID strings.
   * Stored as a `uuid.UUID` object internally.

4. **💳 PaymentCardNumber**

   * Validates credit/debit card numbers.
   * Includes length + checksum checks.

5. **📅 Date & Time Types**

   * `PastDate`, `FutureDate`, `AwareDatetime` → strict temporal validation.
   * Also includes types for IPs, colors, secrets, etc.


---
